Copyright © 2023, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

### MLFlow Preamble

In order to import MLFlow models into SAS Model Manager, there are a few lines that need to be included in the MLFlow script. First of all, include the infer_signature function in the import statements:

```
from mlflow.models.signature import infer_signature
```

Next, adjust any data columns which are not valid Python variable names:

```
import pandas as pd
data = pd.read_csv("YOUR-DATA-GOES-HERE.DATA")
data.columns = data.columns.str.replace("\W|^(?=\d)", "_", regex=True)
```

Include the signature inference after any paramater logging is defined and include a signature argument in the model logging:

```
import mlflow
signature = infer_signature(data, model.predict(data))
mlflow.sklearn.log_model(model, "model", signature=signature)
```

### Python Package Imports

In [1]:
# Pathing support
from pathlib import Path

# sasctl interface for importing models
import sasctl.pzmm as pzmm
from sasctl import Session

### Collect MLFlow Model Details

In [2]:
mlflow_model_path = Path("~/MLFlow/mlflow/examples/sklearn_elasticnet_wine/mlruns/0/7100d4e854224371b39c4a9589317121/artifacts/model")
metadata_dict, inputs_dict, outputs_dict = pzmm.MLFlowModel.read_mlflow_model_file(mlflow_model_path)

In [3]:
metadata_dict

{'python_version': '3.7.4',
 'serialization_format': 'cloudpickle',
 'run_id': '7100d4e854224371b39c4a9589317121',
 'model_path': 'model.pkl',
 'mlflowPath': WindowsPath('~/MLFlow/mlflow/examples/sklearn_elasticnet_wine/mlruns/0/7100d4e854224371b39c4a9589317121/artifacts/model')}

In [4]:
inputs_dict

[{'name': 'fixedacidity', 'type': 'double'},
 {'name': 'volatileacidity', 'type': 'double'},
 {'name': 'citricacid', 'type': 'double'},
 {'name': 'residualsugar', 'type': 'double'},
 {'name': 'chlorides', 'type': 'double'},
 {'name': 'freesulfurdioxide', 'type': 'double'},
 {'name': 'totalsulfurdioxide', 'type': 'double'},
 {'name': 'density', 'type': 'double'},
 {'name': 'pH', 'type': 'double'},
 {'name': 'sulphates', 'type': 'double'},
 {'name': 'alcohol', 'type': 'double'}]

In [5]:
outputs_dict

[{'type': 'tensor', 'tensor-spec': {'dtype': 'float64', 'shape': [-1]}}]

### Register Model in SAS Model Manager with pzmm

In [6]:
model_prefix = "MLFlowModel"
zip_folder = Path.cwd() / "data/MLFlowModels/Model1/"
pzmm.PickleModel.pickle_trained_model(
    model_prefix=model_prefix, 
    pickle_path=zip_folder, 
    mlflow_details=metadata_dict
)

In [7]:
pzmm.JSONFiles.write_var_json(inputs_dict, is_input=True, json_path=zip_folder)
pzmm.JSONFiles.write_var_json(outputs_dict, is_input=False, json_path=zip_folder)

inputVar.json was successfully written and saved to ~\data\MLFlowModels\Model1\inputVar.json
outputVar.json was successfully written and saved to ~\data\MLFlowModels\Model1\outputVar.json


In [8]:
# Write model properties to a json file
pzmm.JSONFiles.write_model_properties_json(
    model_name=model_prefix,
    model_desc="MLFlow Model",
    target_variable="tensor",
    target_values=["1", "0"],
    json_path=zip_folder,
    modeler="sasdemo"
)

# Write model metadata to a json file
pzmm.JSONFiles.write_file_metadata_json(model_prefix, json_path=zip_folder)

ModelProperties.json was successfully written and saved to ~\data\MLFlowModels\Model1\ModelProperties.json
fileMetadata.json was successfully written and saved to ~\data\MLFlowModels\Model1\fileMetadata.json


In [9]:
import getpass
username = getpass.getpass()
password = getpass.getpass()
host = "demo.sas.com"
sess = Session(host, username, password, protocol="http")

In [10]:
from sklearn.linear_model import ElasticNet
model = pzmm.ImportModel.import_model(
    model_files=zip_folder, 
    model_prefix=model_prefix, 
    input_data=inputs_dict,
    project="MLFlowProject",
    score_metrics=["tensor"],
    pickle_type=metadata_dict["serialization_format"],
    predict_method=[ElasticNet.predict, [int]],
    overwrite_model=True,
    model_file_name="MLFlowTest.pickle"
)

Model score code was written successfully to ~\data\MLFlowModels\Model1\score_MLFlowModel.py and uploaded to SAS Model Manager.
All model files were zipped to ~\data\MLFlowModels\Model1.


~\site-packages\sasctl\pzmm\import_model.py:45: UserWarning: No project with the name or UUID MLFlowProject was found.
  warn(f"No project with the name or UUID {project} was found.")


A new project named MLFlowProject was created.
Model was successfully imported into SAS Model Manager as MLFlowModel with the following UUID: c7247696-0b3d-479a-9f24-adc24ce04273.
